In [36]:

from datetime import datetime as dt
import git
import numpy as np
import pandas as pd
import warnings
import math 
import plotly.express as px
import plotly.graph_objects as go
from dash import MATCH, Dash, Input, Output, callback, dcc, html
from vehicle_stream_pipeline import utils

warnings.filterwarnings("ignore")


In [3]:
repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")

# fetch data (here we can automate it)
df_stops = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="MoDstops"
)

df_edges = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="Liste 2022"
)

df_edges.rename(columns={"Start #": "start_id", "Ende #": "end_id"}, inplace=True)

rides_df = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")
rides_df = rides_df[(rides_df["state"] == "completed")]
rides_df["scheduled_to"] = pd.to_datetime(rides_df["scheduled_to"])




In [4]:
start_date = min(rides_df["scheduled_to"])
end_date = max(rides_df["scheduled_to"])

print(start_date)
print(end_date)

2021-07-01 07:30:00
2022-05-31 22:28:43


In [5]:

date_range = utils.get_date_range(start_date,end_date)

data_range_len = len(date_range)
print(date_range[10])


(2022, 5)


In [123]:
total_sim_rides = 50000 # will be filtered later
sim_rides = math.ceil(total_sim_rides/data_range_len)
new_rides_all = pd.DataFrame(columns=rides_df.columns)
for (year, month) in date_range:
            new_rides = utils.generateRideSpecs(
                rides_df,
                df_stops,
                df_edges,
                sim_rides,
                month,
                year,
            )
            new_rides_all = pd.concat([new_rides, new_rides_all])

new_rides_all["simulated"] = True
rides_df["simulated"] = False

In [122]:
new_rides_all['month'] = pd.to_datetime(new_rides_all['scheduled_to'])
new_rides_all['month'] = new_rides_all['month'].apply(lambda x: x.month)
new_rides_all['month'].value_counts()

5     1676
4     1309
3     1157
2      956
1      763
11     649
12     649
10     527
7      446
8      439
9      431
Name: month, dtype: int64

In [15]:
new_rides_all.shape

(50006, 53)

In [16]:
sim_rides_all_1

,arriving_push,vehicle_arrived_at,created_at,scheduled_to,dispatched_at,pickup_at,pickup_arrival_time,dropoff_at,dropoff_eta,pickup_first_eta,shortest_ridetime
0,2022-05-01 19:23:40.822432655,2022-05-01 19:23:02,2022-05-01 19:22:30,2022-05-01 19:22:30,2022-05-01 19:22:30,2022-05-01 19:24:04,32,2022-05-01 19:46:42,2022-05-01 19:49:39,2022-05-01 19:25:30,291.48
1,2022-05-02 11:07:43.000000000,2022-05-02 11:08:12,2022-05-01 16:04:41,2022-05-02 08:40:00,2022-05-02 08:32:00,2022-05-02 11:08:22,9372,2022-05-02 11:25:29,2022-05-02 11:25:37,2022-05-02 11:05:53,302.64
2,2022-05-02 07:59:39.000000000,2022-05-02 08:01:47,2022-05-01 18:29:48,2022-05-02 07:20:00,2022-05-02 07:12:00,2022-05-02 08:03:05,2987,2022-05-02 08:10:01,2022-05-02 08:13:01,2022-05-02 08:04:18,737.88
3,2022-05-01 19:29:19.000000000,2022-05-01 19:39:00,2022-05-01 19:24:36,2022-05-01 19:24:36,2022-05-01 19:24:36,2022-05-01 19:39:16,864,2022-05-01 19:45:41,2022-05-01 19:42:11,2022-05-01 19:33:03,216.00
4,2022-05-01 22:59:10.000000000,2022-05-01 23:04:18,2022-05-01 22:52:43,2022-05-01 22:52:43,2022-05-01 22:52:43,2022-05-01 23:06:25,695,2022-05-01 23:13:43,2022-05-01 23:17:13,2022-05-01 23:08:53,244.92


In [104]:
drones_activated="0",
pickup_address="Mandelgasse"
dropoff_address="Rathaus"
radius=500
sim_rides=5000




In [105]:
rides_df_1 = rides_df.copy()
sim_rides_all_1 = new_rides_all.copy()
route_information = ""
ridetime = "Average time to destination: 77 days"

pickup_address = utils.find_id_for_name(pickup_address, df_stops)
dropoff_address = utils.find_id_for_name(dropoff_address, df_stops)

start_date = dt.strptime("2022-03-01", "%Y-%m-%d")
end_date = dt.strptime("2022-05-31", "%Y-%m-%d")    

if sim_rides != 0:
        sim_rides_all_1 = sim_rides_all_1[
            (sim_rides_all_1["scheduled_to"] > start_date)
            & (sim_rides_all_1["scheduled_to"] < end_date)
        ]
        sim_rides_all_1.sample(n=sim_rides)
        new_rides_all_1 = pd.concat([rides_df_1, sim_rides_all_1])

else:
    new_rides_all_1 = rides_df_1

rides_df_filterd = new_rides_all_1[
    (new_rides_all_1["scheduled_to"] > start_date)
    & (new_rides_all_1["scheduled_to"] < end_date)
]

rides_df_filterd

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index,simulated,month
10330,c65520ff-9844-4dac-a562-f1f91657b2d9,d2a75ace-8b9f-4b32-9914-85e434ce9019,1582.0,1.0,2.88,2.88,2.88,False,STANDARD,1002,1008,completed,NaN,2022-02-28 23:45:25,2022-03-01 07:00:00,2022-03-01 06:52:00,66.0,2022-03-01 06:52:18,2022-03-01 06:53:06,2022-03-01 06:55:00,2022-03-01 06:53:17,2022-03-01 07:01:04,2022-03-01 06:57:33,2022-03-01 06:58:08,2022-03-01 07:05:50,2022-03-01 06:59:11,2022-03-01 06:59:13,-132.0,-114.0,267.0,98.0,-16.0,190.00,-206.00,0.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wochenkarte Jedermann,086020885,NaN,NaN,NaN,NaN,False,NaN
10331,1bd70a56-3a67-4487-bc60-7d770c6e9e60,db131c24-92c7-4a64-b05e-ab8d96ad8458,1159.0,1.0,2.66,2.66,2.66,False,STANDARD,3001,1009,completed,11269.0,2022-03-01 07:47:05,2022-03-01 07:47:05,2022-03-01 07:47:05,693.0,2022-03-01 07:51:56,2022-03-01 07:58:38,2022-03-01 07:50:05,2022-03-01 07:55:28,2022-03-01 07:59:38,2022-03-01 07:58:40,2022-03-01 08:00:20,2022-03-01 08:04:42,2022-03-01 08:05:23,2022-03-01 08:05:24,222.0,513.0,2.0,403.0,916.0,139.00,777.00,2.90,Kein Signalschild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
10332,15cca468-f90f-4781-b930-83a105072fd4,b5ff065c-fedd-4102-9e7d-916330bb79ec,4800.0,1.0,4.64,4.64,4.64,False,STANDARD,5001,1008,completed,NaN,2022-02-28 18:02:19,2022-03-01 08:30:00,2022-03-01 08:22:00,44.0,2022-03-01 08:22:02,2022-03-01 08:22:44,2022-03-01 08:25:00,2022-03-01 08:23:33,2022-03-01 08:23:44,2022-03-01 08:23:30,2022-03-01 08:33:36,2022-03-01 08:33:43,2022-03-01 08:30:45,2022-03-01 08:30:46,-138.0,-136.0,46.0,435.0,299.0,576.00,-277.00,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
10336,d0fb8617-cc8e-497e-80cd-a9d442da16a1,817aaaf0-d5e5-4838-b246-452fad2490ef,4800.0,1.0,4.64,2.49,2.49,False,VRN,1008,4025,completed,11273.0,2022-03-01 09:10:15,2022-03-01 09:10:15,2022-03-01 09:10:15,45.0,2022-03-01 09:10:17,2022-03-01 09:11:00,2022-03-01 09:13:15,2022-03-01 09:11:30,2022-03-01 09:13:11,2022-03-01 09:13:20,2022-03-01 09:19:40,2022-03-01 09:21:21,2022-03-01 09:21:05,2022-03-01 09:21:07,-137.0,-135.0,140.0,465.0,330.0,576.00,-246.00,0.81,Kein Signalschild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rhein-Neckar-Ticket,1641654,NaN,NaN,NaN,NaN,False,NaN
10341,318f0867-9ec8-4e6b-8cb4-8e58f841d203,c2e95a22-7e44-41cf-8175-f59bfc3866fb,2284.0,1.0,3.27,3.27,3.27,False,STANDARD,1007,6003,completed,NaN,2022-03-01 09:07:48,2022-03-01 09:40:00,2022-03-01 09:32:00,468.0,2022-03-01 09:33:51,2022-03-01 09:39:48,2022-03-01 09:35:00,2022-03-01 09:37:47,2022-03-01 09:40:48,2022-03-01 09:40:17,2022-03-01 09:43:46,2022-03-01 09:46:45,2022-03-01 09:47:16,2022-03-01 09:47:17,177.0,288.0,29.0,419.0,707.0,274.00,433.00,1.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4531,1659954961-4531,4531-1659954961,5237,2.0,NaN,NaN,NaN,False,BAHN_CARD,4025,1007,completed,NaN,2022-02-28 18:06:59,2022-03-01 07:00:00,2022-03-01 06:52:00,16287,2022-03-01 11:21:20,2022-03-01 11:23:27,2022-03-01 06:55:00,2022-03-01 11:18:55,2022-03-01 11:24:54,2022-03-01

In [106]:
rides_df_filterd[(rides_df_filterd["pickup_address"]==1001) & (rides_df_filterd["dropoff_address"]==1005)].head(5)

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index,simulated,month
17323,0d120dca-a4b4-4ac1-9244-7d6c9c144e1e,729c1dee-ab62-43a2-abf9-5e6320fb2338,555.0,1.0,2.55,2.55,2.55,False,STANDARD,1001,1005,completed,16198.0,2022-05-19 16:13:20,2022-05-19 16:13:20,2022-05-19 16:13:20,1825.0,2022-05-19 16:41:02,2022-05-19 16:43:45,2022-05-19 16:16:20,2022-05-19 16:53:16,2022-05-19 16:44:45,2022-05-19 16:43:46,2022-05-19 16:58:00,2022-05-19 16:50:25,2022-05-19 16:45:45,2022-05-19 16:45:46,-17.0,1645.0,1.0,119.0,1764.0,67.00,1697.00,1.78,Rotes Signalschild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
189,1659955102-189,189-1659955102,5200,1.0,NaN,NaN,NaN,False,BAHN_CARD,1001,1005,completed,NaN,2022-05-02 18:47:12,2022-05-02 18:47:12,2022-05-02 18:47:12,866,2022-05-02 18:57:58,2022-05-02 19:01:38,2022-05-02 18:50:12,2022-05-02 19:01:37,2022-05-02 19:03:44,2022-05-02 19:02:33,2022-05-02 19:12:01,2022-05-02 19:05:13,2022-05-02 19:04:55,NaN,40.0,686.0,55.0,142.0,828.0,624.00,204.00,0.23,Kein Signalschild,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5.0
1816,1659955102-1816,1816-1659955102,6539,3.0,NaN,NaN,NaN,False,STANDARD,1001,1005,completed,NaN,2022-05-13 12:41:22,2022-05-13 19:40:00,2022-05-13 19:32:00,11215,2022-05-13 22:35:47,2022-05-13 22:38:55,2022-05-13 19:35:00,2022-05-13 22:38:19,2022-05-13 22:38:19,2022-05-13 22:41:23,2022-05-13 22:51:23.680000,2022-05-13 22:42:22,2022-05-13 22:43:15,NaN,8.0,11035.0,148.0,112.0,11147.0,784.68,10362.32,0.14,Gelbes Signalschild,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5.0
2695,1659955102-2695,2695-1659955102,647,1.0,NaN,NaN,NaN,False,STANDARD,1001,1005,completed,NaN,2022-05-19 16:28:20,2022-05-19 16:28:20,2022-05-19 16:28:20,403,2022-05-19 16:31:20,2022-05-19 16:35:03,2022-05-19 16:31:20,2022-05-19 16:33:47,2022-05-19 16:37:46,2022-05-19 16:35:57,2022-05-19 16:35:04.640000,2022-05-19 16:39:49,2022-05-19 16:38:39,NaN,43.0,223.0,54.0,162.0,385.0,77.64,307.36,2.09,Kein Signalschild,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5.0
3509,1659955102-3509,3509-1659955102,3420,2.0,NaN,NaN,NaN,False,VRN,1001,1005,completed,NaN,2022-05-25 19:40:31,2022-05-25 19:40:31,2022-05-25 19:40:31,823,2022-05-25 19:49:21,2022-05-25 19:54:14,2022-05-25 19:43:31,2022-05-25 19:53:02,2022-05-25 19:53:02,2022-05-25 19:54:27,2022-05-25 19:59:52.400000,2022-05-25 20:05:38,2022-05-25 19:56:44,NaN,113.0,643.0,13.0,137.0,780.0,410.40,369.60,0.33,None,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5.0


In [107]:
rides_df_filterd.columns

Index(['id', 'user_id', 'distance', 'number_of_passenger', 'price_operations',
       'price_offer', 'price_payed', 'free_ride', 'payment_type',
       'pickup_address', 'dropoff_address', 'state', 'created_from_offer',
       'created_at', 'scheduled_to', 'dispatched_at', 'pickup_arrival_time',
       'arriving_push', 'vehicle_arrived_at', 'earliest_pickup_expectation',
       'pickup_first_eta', 'pickup_eta', 'pickup_at', 'dropoff_first_eta',
       'dropoff_eta', 'dropoff_at', 'updated_at', 'arrival_deviation',
       'waiting_time', 'boarding_time', 'ride_time', 'trip_time',
       'shortest_ridetime', 'delay', 'longer_route_factor',
       'arrival_indicator', 'rating', 'rating_puenktlichkeit',
       'rating_sauberkeit', 'rating_fahrer', 'rating_find_modstop',
       'rating_other_comments', 'cancellation_reason', 'cancellation_comment',
       'bahn_card_number', 'year_card_type', 'year_card_number', 'canceled_at',
       'rating_question_one', 'rating_question_two', 'index', 's

In [108]:
# print(len(rides_df_filterd))
# if default parameters None, do nothing else get shortest ride of function call
if pickup_address is not None or dropoff_address is not None:

    drives_without_drones_1 = utils.calculate_drives(
        rides_df_filterd[rides_df_filterd["simulated"]==True], start_date, end_date
    )
    drives_without_drones_2 = utils.calculate_drives(
        rides_df_filterd[rides_df_filterd["simulated"]==False], start_date, end_date
    )

    drives_without_drones = utils.calculate_drives(
        rides_df_filterd, start_date, end_date
    )

In [109]:
drives_without_drones[(drives_without_drones["pickup_address"]==1001) & (drives_without_drones["dropoff_address"]==1005)]

,pickup_address,dropoff_address,number_of_drives,waiting_time,avg_ride_time,avg_time_to_destination
1,1001,1005,24,3.833333,0.001355,3.834688


In [110]:
drives_without_drones_1[(drives_without_drones_2["pickup_address"]==1001) & (drives_without_drones_1["dropoff_address"]==1005)]

,pickup_address,dropoff_address,number_of_drives,waiting_time,avg_ride_time,avg_time_to_destination
1,1001,1005,23,4.0,0.001354,4.001354


In [111]:
drives_without_drones_2[(drives_without_drones_2["pickup_address"]==1001) & (drives_without_drones_2["dropoff_address"]==1005)]

,pickup_address,dropoff_address,number_of_drives,waiting_time,avg_ride_time,avg_time_to_destination
0,1001,1005,1,92.0,0.001377,92.001377


In [112]:
hotspots = utils.get_hotspots(df_edges, drives_without_drones)
    # hotspots = [spot[0] for spot in hotspots]
    # hotspots = [1008, 4025, 1005, 1009, 1007, 12007, 7001, 6004, 1010, 11017]

drone_spots = [15011, 13001, 2002, 11007, 4016, 1002, 3020, 9019, 9005]

df_stops_drones = df_stops[df_stops["MoDStop Id"].isin(drone_spots)]


layers = []

graph_without_drones = utils.calculate_graph(drives_without_drones)
path, shortest_time = utils.get_shortest_ride(
    pickup_address, dropoff_address, graph_without_drones
)
route_information = utils.get_route_information(
    drives_without_drones, path, df_stops
)


ridetime = f"Average time to destination: {round(shortest_time, 2)} days"
# get lat and lon for traces for each stop in the returend list
latitudes, longitudes = utils.get_geo_cordinates_for_path(df_stops, path)



In [113]:
ridetime

'Average time to destination: 3.83 days'

In [114]:
pickup_counts = (
rides_df_filterd.groupby("pickup_address")
.size()
.to_frame("number_of_pickups")
.reset_index()
)
dropoff_counts = (
rides_df_filterd.groupby("dropoff_address")
.size()
.to_frame("number_of_dropoffs")
.reset_index()
)

pickup_counts["pickup_address"] = pickup_counts["pickup_address"].astype(int)
dropoff_counts["dropoff_address"] = dropoff_counts["dropoff_address"].astype(int)

df_stops_1 = pd.merge(
df_stops, pickup_counts, left_on="MoDStop Id", right_on="pickup_address"
).drop("pickup_address", axis=1)
df_stops_1 = pd.merge(
df_stops_1, dropoff_counts, left_on="MoDStop Id", right_on="dropoff_address"
).drop("dropoff_address", axis=1)

if drones_activated == "1":
    df_stops_1["is_drone_spot_color"] = np.where(
        df_stops_1["MoDStop Id"].isin(drone_spots), "purple", "blue"
    )

    df_stops_1["is_drone_spot_size"] = np.where(
        (df_stops_1["MoDStop Id"].isin(hotspots))
        | (df_stops_1["MoDStop Id"].isin(drone_spots)),
        12,
        6,
    )

else:
    df_stops_1["is_drone_spot_color"] = np.where(
        df_stops_1["MoDStop Id"].isin(drone_spots), "blue", "blue"
    )

    df_stops_1["is_drone_spot_size"] = np.where(
        (df_stops_1["MoDStop Id"].isin(hotspots)), 12, 6
    )

print()

In [115]:
fig = go.Figure(
    go.Scattermapbox(
        mode="markers",
        lat=df_stops_1["MoDStop Lat"],
        lon=df_stops_1["MoDStop Long"],
        # [df_stops_1['MoDStop Name'], df_stops_1['MoDStop Id']]
        hovertext=df_stops_1["MoDStop Name"],
        hoverinfo="text",
        marker={
            "color": df_stops_1["is_drone_spot_color"],
            "size": df_stops_1["is_drone_spot_size"],
        },
        showlegend=False,
    )
)
#
fig.add_trace(
    go.Scattermapbox(
        lon=longitudes,
        lat=latitudes,
        mode="markers+lines",
        line=dict(width=2, color="red"),
        # [df_stops_1['MoDStop Name'], df_stops_1['MoDStop Id']]
        # hovertext=df_stops_1["MoDStop Name"],
        hoverinfo="skip",
        opacity=1,
        showlegend=False,
    )
)
fig = fig.update_layout(
    mapbox_layers=layers,
    mapbox_style="open-street-map",
    mapbox_zoom=11,
    mapbox_center={"lat": 49.3517, "lon": 8.13664},
    plot_bgcolor="rgba(0, 0, 0, 0)",
    paper_bgcolor="rgba(0, 0, 0, 0)",
    margin=dict(l=20, r=20, t=20, b=20),
)


In [116]:
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'hovertext': array(['Mandelgasse', 'Hauptfeuerwache', 'Kindergarten St. Marien', ...,
                                  'Altes Schulhaus Lachen', 'Duttweiler Dorfplatz', 'Festhalle Geinsheim'],
                                 dtype=object),
              'lat': array([49.35178 , 49.353733, 49.355745, ..., 49.322493, 49.299327, 49.304204]),
              'lon': array([8.129   , 8.131552, 8.134341, ..., 8.196059, 8.211596, 8.257313]),
              'marker': {'color': array(['blue', 'blue', 'blue', ..., 'blue', 'blue', 'blue'], dtype=object),
                         'size': array([6, 6, 6, ..., 6, 6, 6])},
              'mode': 'markers',
              'showlegend': False,
              'type': 'scattermapbox'},
             {'hoverinfo': 'skip',
              'lat': [49.35178, 49.354022],
              'line': {'color': 'red', 'width': 2},
              'lon': [8.129, 8.135143],
              'mode': 'markers+lines',
              'opacity': 1,
              'showlegend': False,
              'type': 'scattermapbox'}],
    'layout': {'mapbox': {'center': {'lat': 49.3517, 'lon': 8.13664}, 'style': 'open-street-map', 'zoom': 11},
               'margin': {'b': 20, 'l': 20, 'r': 20, 't': 20},
               'paper_bgcolor': 'rgba(0, 0, 0, 0)',
               'plot_bgcolor': 'rgba(0, 0, 0, 0)',
               'template': '...'}
})

In [131]:
drones_activated="0",
pickup_address="Mandelgasse"
dropoff_address="Schütt"
radius=500
sim_rides=10000

rides_df_1 = rides_df.copy()
sim_rides_all_1 = new_rides_all.copy()
route_information = ""
ridetime = "Average time to destination: 77 days"

pickup_address = utils.find_id_for_name(pickup_address, df_stops)
dropoff_address = utils.find_id_for_name(dropoff_address, df_stops)

start_date = dt.strptime("2022-03-01", "%Y-%m-%d")
end_date = dt.strptime("2022-05-31", "%Y-%m-%d")    

if sim_rides != 0:
    sim_rides_all_1 = sim_rides_all_1[
        (sim_rides_all_1["scheduled_to"] > start_date)
        & (sim_rides_all_1["scheduled_to"] < end_date)
    ]
    sim_rides_all_1.sample(n=sim_rides)
    new_rides_all_1 = pd.concat([rides_df_1, sim_rides_all_1])

else:
    new_rides_all_1 = rides_df_1

rides_df_filterd = new_rides_all_1[
    (new_rides_all_1["scheduled_to"] > start_date)
    & (new_rides_all_1["scheduled_to"] < end_date)
]

# print(len(rides_df_filterd))
# if default parameters None, do nothing else get shortest ride of function call
if pickup_address is not None or dropoff_address is not None:

    drives_without_drones = utils.calculate_drives(
        rides_df_filterd, start_date, end_date
    )

    hotspots = utils.get_hotspots(df_edges, drives_without_drones)
    # hotspots = [spot[0] for spot in hotspots]
    # hotspots = [1008, 4025, 1005, 1009, 1007, 12007, 7001, 6004, 1010, 11017]

    drone_spots = [15011, 13001, 2002, 11007, 4016, 1002, 3020, 9019, 9005]

    df_stops_drones = df_stops[df_stops["MoDStop Id"].isin(drone_spots)]

    if drones_activated == "0":
        layers = []

        graph_without_drones = utils.calculate_graph(drives_without_drones)
        path, shortest_time = utils.get_shortest_ride(
            pickup_address, dropoff_address, graph_without_drones
        )
        route_information = utils.get_route_information(
            drives_without_drones, path, df_stops
        )
    # else:
    #     layers = utils.create_circles_around_drone_spots(df_stops_drones, radius)

    #     drives_with_drones = utils.add_drone_flights(
    #         df_edges, drives_without_drones, drone_spots=drone_spots, radius=radius
    #     )
    #     graph_with_drones = utils.calculate_graph(drives_with_drones)
    #     path, shortest_time = utils.get_shortest_ride(
    #         pickup_address, dropoff_address, graph_with_drones
    #     )
    #     route_information = utils.get_route_information(
    #         drives_with_drones, path, df_stops
    #     )

    ridetime = f"Average time to destination: {round(shortest_time, 2)} days"
    # get lat and lon for traces for each stop in the returend list
    latitudes, longitudes = utils.get_geo_cordinates_for_path(df_stops, path)

print(shortest_time)


0.0009044312169312169
